# Using  EAGO with a script-defined problem:  
### *Kinetic parameter estimation with implicit Euler integration*

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

EAGO includes a specialized series of routines which can solve certain optimization problems by relaxing implicit functions directly without the introduction of additional auxilliary variables. The implicit functions, $\mathbf{x(p)}$ are defined by a series of equalities $\mathbf{h}(\mathbf{x},\mathbf{p}) = \mathbf{0}$ on some domain $(X \times P)$ allow for the problem to be treated implicitly using the below form. While exact evaluations of the objective function and constraints in this form are difficult, relaxations used in global optimization are much easier to calculate. In some cases, this problem may be much easier to solve when formulated in this manner due to a significant reduction in decision variable space (in cases where $n_x >> n_p$). 

$
\begin{align}\label{form:2}
    f^*=&\min_{\mathbf p \in P}f(\mathbf x(\mathbf p),\mathbf p)\\
    &{\rm s.t.}\; \mathbf{g(x(p),p)\le 0}.
\end{align}
$

## Problem Setup and Motivation

In **nlpopt_explicit_kinetic.ipynb** we described how the kinetic parameter estimation problem can be solved in EAGO using an explicit script formulation that arises from explicit Euler discretization. In this notebook, we'll illustrate how the implicit Euler discretization can be solved using EAGO's unique implicit global optimizer. Consider the implicit euler discretization of the problem estimation problem described in [1-4]:

$
\begin{align} 
x_A^{i} &= x_A^{i-1} + \Delta t \left(k_1 x_Y^{i} x_Z^{i} - C_{O2}(k_{2f}+k_{3f})x_A^i + \frac{k_{2f}}{K_2}x^i_D + \frac{k_{3f}}{K_3}x^i_B - k_5 (x_A^i)^2 \right) \\
x_B^{i} &= x_B^{i-1} + \Delta t \left(k_{3f}C_{O2}x_A^i - \left(\frac{k_{3f}}{K_3} + k_4\right)x_B^i\right) \\
x_D^{i} &= x_D^{i-1} + \Delta t \left(k_{2f}C_{O2}x_A^i - \frac{k_{2f}}{K_2} x_D^i\right) \\
x_Y^{i} &= x_Y^{i-1} + \Delta t \left(-k_{1s} x_Y^i x_Z^i \right) \\
x_Z^{i} &= x_Z^{i-1} + \Delta t \left(-k_{1s} x_Y^i x_Z^i\right) \label{ex:ImplicitEuler1}
\end{align}
$

## Formulate objective
We formulate the objective as a regular function of the state variables $\mathbf{x}$ and the control variables $\mathbf{p}$.

In [ ]:
using EAGO, JuMP, DataFrames, CSV

data = CSV.read('kinetic_intensity_data.csv')
xbnds = CSV.read('implicit_variable_bounds.csv')
xl, xu = xbnds[:lower], xbnds[:upper]

pl = [10.0  10.0  0.001]
pu = [1200.0  1200.0  40.0]

# Define function for intensity
I(x,y,z) = x + (2/21)*y + (2/21)*z

# Objective and constraint functions are defined
function f(x,p)
    SSE = 0.0
    for i=1:200
        SSE += (I(x[5i-4],x[5i-3],x[5i-2]) - data[:intensity][i])^2
    end
    return SSE
end

## Define the implicit function & jacobian of state function

We now define the function $\mathbf{h}(\mathbf{x},\mathbf{p}) = \mathbf{0}$ which defines the state variable. EAGO's implicit solver assumes this function is defined in place as a function of the output variables, state variables, and control variables, respectively.

In [2]:
# Specifies equalities function that define implicit function
function h!(out,x,p)

    fill!(out,zero(typeof(p)))    # reset the storage array
    out[4] = 0.4; out[5] = 140         # sets initial condition
    
    # sets known parameter values
    T = 273; delT = 0.01; cO2 = 2e-3; k1 = 53; k1s = k1*1e-6
    K2 = 46exp(-6500/T-18); K3 = 2*K2;  
   
    for i in 1:200
        # Calculation of intermediate terms
        termA = delT*(k1s*x[5i+4]*x[5i+5]-cO2*(p[1]+p[2])*x[5i+1])
        termB = delT*(p[1]*x[5i+1]/K2+p[2]*x[5i+2]/K3-k5*x[5i+1]^2)
        termC = p[2]*cO2*x[5i+1]-(p[2]/K3+p[3])x[5i+2]
        termD = p[1]*cO2*x[5i+1]-p[1]*x[5i+3]/K2
    
        # Calculation of next time step
        out[5i+1] =  x[5i-4] - x[5i+1] + termA + termB
        out[5i+2] = x[5i-3] - x[5i+2] + delT*termC
        out[5i+3] = x[5i-2] - x[5i+3] + delT*termD
        out[5i+4] = x[5i-1] - x[5i+4] + delT*(-k1s*x[5i+4]*x[5i+5])
        out[5i+5] = x[5i] - x[5i+5] + delT*(-k1s*x[5i+4]*x[5i+5])
    end
    
end

h! (generic function with 1 method)

We now define the the jacobian of the state function *w.r.t* to state variables only. This function must be supplied in place as this allows better integration with different linear algebra libraries. **EAGO** will use an automated differentiation library to calculate this function if the user does not provide it. However, some performance improvement is expected when a user-defined function is supplied. Typically, this improvement is on the order of 3-10x.

In [ ]:
# Specifies equalities function that define implicit function
function hj!(out,x,p)

    fill!(out,zero(typeof(p[1])))    # reset the storage array
    out[4] = 0.4; out[5] = 140       # sets initial condition
    
    # sets known parameter values
    T = 273; delT = 0.01; cO2 = 2e-3; k1 = 53; k1s = k1*1e-6
    K2 = 46exp(6500/T-18); K3 = 2*K2;  
   
    for i in 1:200
        # Calculation of intermediate terms (not used)
        #termA = delT*(k1*x[5i+4]*x[5i+5]-cO2*(p[1]+p[2])*x[5i+1])
        #termB = delT*(p[1]*x[5i+1]/K2+p[2]*x[5i+2]/K3-k5*x[5i+1]^2)
        #termC = p[2]*cO2*x[5i+1]-(p[2]/K3+p[3])x[5i+2]
        #termD = p[1]*cO2*x[5i+1]-p[1]/K2
    
        # partials of  x[5i-4] - x[5i+1] + delT*(k1s*x[5i+4]*x[5i+5]-cO2*(p[1]+p[2])*x[5i+1]) 
        #                                + delT*(p[1]*x[5i+1]/K2+p[2]*x[5i+2]/K3-k5*x[5i+1]^2)
        out[5i+1,5i-4] =  one(p[1])
        out[5i+1,5i+1] = -one(p[1]) - delT*cO2*(p[1]+p[2]) + (delT/K2)*p[1] - (2.0*delT*k5)*x[5i+1]
        out[5i+1,5i+2] = delT*p[2]/K3
        out[5i+1,5i+4] = delT*k1s*x[5i+5]
        out[5i+1,5i+5] = delT*k1s*x[5i+4]
        
        # partials of  x[5i-3] - x[5i+2] + delT*(p[2]*cO2*x[5i+1]-(p[2]/K3+p[3])x[5i+2])
        out[5i+2, 5i-3] = one(p[1])
        out[5i+2, 5i+1] = (delT*cO2)*p[2]
        out[5i+2, 5i+2] = -one(p[1])-delT*(p[2]/K3+p[3])
        
        # partials of  x[5i-2] - x[5i+3] + delT*(p[1]*cO2*x[5i+1]-p[1]/K2)
        out[5i+3, 5i-2] = one(p[1])
        out[5i+3, 5i+1] = (cO2*delT)*p[1]*x[5i+1]
        out[5i+3, 5i+3] = -one(p[1])
        
        # partials of  x[5i-1] - x[5i+4] + delT*(-k1s*x[5i+4]*x[5i+5])
        out[5i+4, 5i-1] = one(p[1])
        out[5i+4, 5i+4] = -one(p[1]) + (-delT*k1s)*x[5i+5]
        out[5i+4, 5i+5] = (-delT*k1s)*x[5i+4]
        
        # partials of  x[5i] - x[5i+5] + delT*(k1*x[5i+4]*x[5i+5]) # TO CHECK
        out[5i+5, 5i] = one(p[1])
        out[5i+5, 5i+4] = (delT*k1)*x[5i+5]
        out[5i+5, 5i+5] = -one(p[1]) + (-delT*k1s)*x[5i+4]
    end
    
end

## Solve the implicit formulation

In [ ]:
var, opt = solve_implicit(f, h!, xl, xu, pl, pu, hj = hj!)

### Reference:
1. 1. J. W. Taylor, et al., Direct measurement of the fast, reversible addition of oxygen to cyclohexadienyl radicals in nonpolar solvents, *J. Phys. Chem. A.*, **2004**, 108, 7193–7203.
2. A. B. Singer, et al., Global dynamic optimization for parameter estimation in chemical kinetics, *J. Phys. Chem A.*, **2006**, 110, 971–976.
3. Mitsos, A. Chachuat, B., & Barton, P.I., McCormick-based relaxations of algorithms, *SIAM Journal on Optimization*, *SIAM*, **2009**, 20(2), 573-601.
4. Stuber, M.D., Scott, J.K., & Barton, P.I.: Convex and concave relaxations of implicit functions. *Optim. Methods Softw.*, **2015** 30(3), 424–460.